# 1. Configuración del ambiente

In [2]:
#Pablo
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

global df_banco, resultados

# 2. Preprocesamiento de datos

# 3. Exploración de datos

# 4. Construcción de los modelos

# 5. Evaluación y Selección de los modelos